# Import Modul

In [1]:
!pip install tensorflowjs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 3.32.0 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.
db-dtypes 1.4.3 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from sklearn.metrics.pairwise import cosine_similarity
from flask import Flask, render_template, request, jsonify
from urllib.parse import urlparse, parse_qs
import os
import tensorflowjs as tfjs


# Data Loading

In [ ]:
DATA_PATH = os.path.join('data', 'data\Dataset Materi.xlsx') 
df = pd.read_excel(DATA_PATH, sheet_name='Final')

# Preprocessing

In [ ]:
df["input_text"] = (
    df["Mata Pelajaran"] + " " +
    df["Materi"] + " " +
    df["Sub Materi"].fillna("") + " " +
    df["Jenjang"].astype(str) 
)
df["output_text"] = df['Jenjang'].astype(str) + " | " + df["Materi"] + " | " + df["Sub Materi"] + " | " + df["Link"]

In [11]:
df["input_text"] = df["input_text"].astype(str)
vectorizer = TextVectorization(output_mode="tf-idf", max_tokens=10000, ngrams=(1,2))
text_ds = tf.data.Dataset.from_tensor_slices(df["input_text"].astype(str)).batch(32)
vectorizer.adapt(text_ds)

X_input = vectorizer(df["input_text"])

# Modeling

In [12]:
input_dim = X_input.shape[1]
input_layer = tf.keras.Input(shape=(input_dim,))

# Encoder
x = tf.keras.layers.Dense(512, activation="relu")(input_layer)
x = tf.keras.layers.Dense(256, activation="relu")(x)
encoded = tf.keras.layers.Dense(64, activation="relu")(x)

# Decoder
x = tf.keras.layers.Dense(256, activation="relu")(encoded)
x = tf.keras.layers.Dense(512, activation="relu")(x)
decoded = tf.keras.layers.Dense(input_dim, activation="linear")(x)



In [13]:
autoencoder = tf.keras.Model(inputs=input_layer, outputs=decoded)
autoencoder.compile(optimizer='adam', loss='mse')

In [14]:
autoencoder.fit(X_input, X_input,
                epochs=100,
                verbose=0)

In [15]:
encoder = tf.keras.Model(inputs=input_layer, outputs=encoded)
embedded_vectors = encoder.predict(X_input)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


In [16]:
# Fungsi Rekomendasi
def get_recommendations(query, df, vectorizer, encoder, embedded_vectors, top_n=5):
    query_vectorized = vectorizer([query])
    query_embedded = encoder.predict(query_vectorized)
    similarities = cosine_similarity(query_embedded, embedded_vectors).flatten()
    most_similar_indices = similarities.argsort()[-top_n:][::-1]
    return df.loc[most_similar_indices, ["Jenjang","Materi", "Sub Materi", "Link", "output_text"]]

# Deploy Model

In [ ]:
encoder.save('model\model.keras')

In [ ]:
encoder.save('model\model.h5')

# Convert Model

In [32]:
!tensorflowjs_converter --input_format=keras model.h5 model

2025-05-26 08:48:22.297097: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748249302.321712    4947 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748249302.328987    4947 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🌲 Try https://ydf.readthedocs.io, the successor of TensorFlow Decision Forests with more features and faster training!
failed to lookup keras version from the file,
    this is likely a weight only file
